In [1]:
import os


In [2]:
%pwd

'e:\\Kidney-Disease-Cassification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Kidney-Disease-Cassification'

In [5]:
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/Ayan7020/Kidney-Disease-Cassification.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']= "Ayan7020"
os.environ['MLFLOW_TRACKING_PASSWORD']="d5eff499316a1a11f849856b10b97203fa492f5a"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_url: str
    params_image_size: list  

In [9]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml,save_json

In [10]:
class ConfigurationManager:
        def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_fiepath = PARAMS_FILE_PATH):
            
            self.config = read_yaml(config_filepath) 
            self.params = read_yaml(params_fiepath)    
            
        def get_evaluation_config(self) ->  EvaluationConfig:
            eval_config = EvaluationConfig(
                path_of_model="artifacts/training/model.h5",
                training_data="artifacts/data_ingestion/CT_KINDEY_DATASET",
                mlflow_url="https://dagshub.com/Ayan7020/Kidney-Disease-Cassification.mlflow",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE
            )  
            
            return eval_config

In [11]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [12]:
class Evaluation:
    def __init__(self,config: EvaluationConfig):
        self.config = config
        
        
    def _valid_generator(self):
        
        self.train_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            image_size=self.config.params_image_size[:-1], 
            validation_split=0.1,
            subset="training",
            seed=123,
            label_mode='categorical'
        )

        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            image_size=self.config.params_image_size[:-1], 
            validation_split=0.2,
            subset="validation",
            seed=123,
            label_mode='categorical'
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)  
    
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()  
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    
    def log_in_to_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],"accuracy":self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")     
        
        

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_in_to_mlflow()

except Exception as e:
   raise e

[2023-12-18 12:16:20,794: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-18 12:16:20,800: INFO: common: yaml file: params.yaml loaded successfully]
Found 12446 files belonging to 4 classes.
Using 11202 files for training.
Found 12446 files belonging to 4 classes.
Using 2489 files for validation.

[2023-12-18 12:16:26,393: WARNING: module_wrapper: From C:\Users\Adil\AppData\Roaming\Python\Python311\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
78/78 [==============================] - 365s 5s/step - loss: 0.0179 - accuracy: 0.9976
[2023-12-18 12:22:29,890: INFO: common: json file saved at: scores.json]


2023/12/18 12:22:34 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\Adil\AppData\Local\Temp\tmpf10aq9nt\model\data\model\assets
[2023-12-18 12:22:38,643: INFO: builder_impl: Assets written to: C:\Users\Adil\AppData\Local\Temp\tmpf10aq9nt\model\data\model\assets]


C:\Users\Adil\AppData\Roaming\Python\Python311\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/12/18 12:23:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
